# Project

## Class

In [9]:
import os
import xml.etree.ElementTree as ET
import cv2
import numpy as np

from sklearn.linear_model import SGDClassifier
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import make_pipeline
from skimage.feature import hog

from sklearn.metrics import confusion_matrix, classification_report

from tqdm import tqdm
import matplotlib.pyplot as plt
import seaborn as sns

import joblib

In [10]:
"""
Dataset is stuctured as:
    ├── dataset
    │   ├── images
    │   ├── labels
    │   ├── meta-data
    │   ├── model.pkl
    │   ├── train_files.txt
    │   ├── val_files.txt
    │   └── voc_labels

    Dataset classes:
        Person
        Head
        Face
        Glasses
        Face-mask-medical
        Face-guard
        Ear
        Earmuffs
        Hands
        Gloves
        Foot
        Shoes
        Safety-vest
        Tools
        Helmet
        Medical-suit
        Safety-suit

    We only care about person class so
    for each image:
        - extract just the "Person" class as positive
        - background as negative features.
"""
       

class HOGPersonDetectorTrain:
    
    def __init__(self):
        # HOG parameters
        self.hog_params = {
            'orientations': 9,
            'pixels_per_cell': (8, 8),
            'cells_per_block': (2, 2),
            'block_norm': 'L2'
        }
        
        # Initialize classifier pipeline
        self.clf = make_pipeline(
            StandardScaler(),
            SGDClassifier(
                loss='hinge',  # Linear SVM
                max_iter=1000, 
                tol=1e-3,
                random_state=42
            )
        )


    def train(self, train_file_path, train_images_path, annotations_path, 
              model_save_path="model.pkl", batch_size=50):
        """Train the SVM classifier with batch processing"""
        if not self.check_paths(train_file_path, train_images_path, annotations_path):
            return False
            
        # Load and prepare training data
        train_images = self.load_image_list(train_file_path)
        print(f"Loaded {len(train_images)} images")
            
        print("Preparing training data...")
        X_train, y_train = self.prepare_training_data(
            train_images_path, 
            train_images, 
            annotations_path,
            batch_size=batch_size
        )
        
        print(f"Total features: {len(X_train)}")
        print(f"Positive samples: {np.sum(y_train == 1)}")
        print(f"Negative samples: {np.sum(y_train == 0)}")
        print(f"Feature dimension: {X_train.shape[1]}")
             
        print("Training classifier...")
        self.clf.fit(X_train, y_train)
    
        print("Saving model...")
        self.save_model(model_save_path)
        print(f"Model saved to {model_save_path}")
        
        return True

    def prepare_training_data(self, image_dir, image_list, annotations_dir, batch_size=50):
        """Extract positive and negative samples from dataset in batches"""
        final_X = []
        final_y = []
        
        # Process images in batches
        num_batches = len(image_list) // batch_size + 1
        
        for batch_idx in range(num_batches):
            start_idx = batch_idx * batch_size
            end_idx = min((batch_idx + 1) * batch_size, len(image_list))
            batch_images = image_list[start_idx:end_idx]
            
            print(f"\nProcessing batch {batch_idx + 1}/{num_batches}")
            batch_X = []
            batch_y = []
            
            for img_file in tqdm(batch_images):
                try:
                    # Load image and annotation
                    img_path = os.path.join(image_dir, img_file)
                    xml_name = os.path.splitext(img_file)[0] + '.xml'
                    xml_path = os.path.join(annotations_dir, xml_name)
                    
                    if not os.path.exists(img_path) or not os.path.exists(xml_path):
                        continue
                        
                    image = cv2.imread(img_path)
                    if image is None:
                        continue
        
                    # Convert to grayscale
                    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
                    
                    # Get person boxes
                    person_boxes = self.get_person_boxes(xml_path)
                    
                    # Extract samples with limited negatives
                    pos_samples, neg_samples = self.extract_samples(gray, person_boxes)
                    
                    # Add to batch data
                    batch_X.extend(pos_samples)
                    batch_y.extend([1] * len(pos_samples))
                    batch_X.extend(neg_samples)
                    batch_y.extend([0] * len(neg_samples))
                    
                except Exception as e:
                    print(f"Error processing {img_file}: {e}")
                    continue
            
            # Convert batch to numpy array and append to final lists
            if batch_X:
                batch_X = np.array(batch_X)
                batch_y = np.array(batch_y)
                
                # Save intermediate results
                temp_path = f"temp_batch_{batch_idx}.npz"
                np.savez(temp_path, X=batch_X, y=batch_y)
                
                print(f"Batch {batch_idx + 1} stats:")
                print(f"Positive samples: {np.sum(batch_y == 1)}")
                print(f"Negative samples: {np.sum(batch_y == 0)}")
                
                # Clear batch data
                del batch_X
                del batch_y
                
            # Load previous batches in chunks
            if batch_idx > 0 and batch_idx % 10 == 0:
                print("\nCombining intermediate batches...")
                for i in range(max(0, batch_idx - 10), batch_idx):
                    temp_path = f"temp_batch_{i}.npz"
                    if os.path.exists(temp_path):
                        data = np.load(temp_path)
                        final_X.extend(data['X'])
                        final_y.extend(data['y'])
                        os.remove(temp_path)
        
        # Load any remaining batches
        for i in range(num_batches):
            temp_path = f"temp_batch_{i}.npz"
            if os.path.exists(temp_path):
                data = np.load(temp_path)
                final_X.extend(data['X'])
                final_y.extend(data['y'])
                os.remove(temp_path)
        
        return np.array(final_X), np.array(final_y)
    
    def extract_samples(self, image, person_boxes, neg_pos_ratio=5):
        """Extract positive and negative samples with consistent size"""
        height, width = image.shape[:2]
        positive_samples = []
        negative_samples = []
        window_size = (64, 128)  # Fixed window size for both positive and negative
        
        # Process positive samples
        for box in person_boxes:
            xmin, ymin, xmax, ymax = box
            # Skip boxes that are too small
            if (ymax - ymin) < 64 or (xmax - xmin) < 32:
                continue
                
            # Adjust aspect ratio to match window size
            box_width = xmax - xmin
            box_height = ymax - ymin
            ar = box_height / box_width
            target_ar = window_size[1] / window_size[0]
            
            if ar > target_ar:
                # Too tall, adjust width
                new_width = box_height / target_ar
                xmin = max(0, xmin - (new_width - box_width)/2)
                xmax = min(width, xmax + (new_width - box_width)/2)
            else:
                # Too wide, adjust height
                new_height = box_width * target_ar
                ymin = max(0, ymin - (new_height - box_height)/2)
                ymax = min(height, ymax + (new_height - box_height)/2)
                
            person_img = image[int(ymin):int(ymax), int(xmin):int(xmax)]
            features = self.extract_hog_features(person_img, target_size=window_size)
            if features is not None:
                positive_samples.append(features)
        
        # Create person mask
        person_mask = np.zeros((height, width), dtype=np.uint8)
        for box in person_boxes:
            xmin, ymin, xmax, ymax = box
            person_mask[ymin:ymax, xmin:xmax] = 1
        
        # Extract negative samples with smaller stride
        stride = 32  # Smaller stride for more samples
        target_negatives = len(positive_samples) * neg_pos_ratio
        
        for y in range(0, height - window_size[1], stride):
            if len(negative_samples) >= target_negatives:
                break
            for x in range(0, width - window_size[0], stride):
                if len(negative_samples) >= target_negatives:
                    break
                    
                window_mask = person_mask[y:y + window_size[1], x:x + window_size[0]]
                overlap = np.sum(window_mask) / (window_size[0] * window_size[1])
                
                # Accept windows with less than 30% overlap
                if overlap < 0.3:
                    window = image[y:y + window_size[1], x:x + window_size[0]]
                    features = self.extract_hog_features(window, target_size=window_size)
                    if features is not None:
                        negative_samples.append(features)
        
        return positive_samples, negative_samples
        
    def load_image_list(self, split_file_path):
        """Load image list from train/val txt file"""
        with open(split_file_path) as f:
            return [line.strip() for line in f.readlines()]

    def get_person_boxes(self, xml_path):
        """Extract person bounding boxes from XML annotation"""
        tree = ET.parse(xml_path)
        root = tree.getroot()
        boxes = []
        
        for obj in root.findall('object'):
            if obj.find('name').text.lower() == 'person':
                bbox = obj.find('bndbox')
                xmin = int(float(bbox.find('xmin').text))
                ymin = int(float(bbox.find('ymin').text))
                xmax = int(float(bbox.find('xmax').text))
                ymax = int(float(bbox.find('ymax').text))
                boxes.append((xmin, ymin, xmax, ymax))
        return boxes

    def save_model(self, save_path):
        """Save the trained model"""
        joblib.dump(self.clf, save_path)

    def check_paths(self, train_file_path, train_images_path, annotations_path):
        """Verify all required paths exist"""
        if not all(os.path.exists(p) for p in [train_file_path, train_images_path, annotations_path]):
            print("One or more required paths do not exist")
            return False
            
        with open(train_file_path) as f:
            image_list = [line.strip() for line in f.readlines()]
            print(f"Found {len(image_list)} images in train file")
            
        return True
    
    def extract_hog_features(self, image, target_size=(128, 64)):
        """Extract HOG features from an image patch"""
        if image is None or image.size == 0:
            return None
            
        try:
            resized = cv2.resize(image, target_size)
            features = hog(
                resized,
                orientations=self.hog_params['orientations'],
                pixels_per_cell=self.hog_params['pixels_per_cell'],
                cells_per_block=self.hog_params['cells_per_block'],
                block_norm=self.hog_params['block_norm'],
                feature_vector=True
            )
            return features
        except Exception as e:
            print(f"Error extracting HOG features: {e}")
            return None
      

## Code

In [11]:
detector = HOGPersonDetectorTrain()

In [12]:
train_file_path = "dataset/train_files.txt"
train_images_path = "dataset/images"
annotations_path = "dataset/voc_labels"
model_save_path = "model.pkl"

detector.train(train_file_path, train_images_path, annotations_path, model_save_path, batch_size=50 )

Found 6479 images in train file
Loaded 6479 images
Preparing training data...

Processing batch 1/130


100%|██████████| 50/50 [00:07<00:00,  6.66it/s]


Batch 1 stats:
Positive samples: 64
Negative samples: 305

Processing batch 2/130


100%|██████████| 50/50 [00:07<00:00,  6.78it/s]


Batch 2 stats:
Positive samples: 78
Negative samples: 385

Processing batch 3/130


...

Batch 129 stats:
Positive samples: 114
Negative samples: 570

Processing batch 130/130


100%|██████████| 29/29 [00:04<00:00,  5.99it/s]


Batch 130 stats:
Positive samples: 43
Negative samples: 215
Total features: 64990
Positive samples: 10920
Negative samples: 54070
Feature dimension: 3780
Training classifier...
Saving model...
Model saved to model.pkl


/usr/local/lib/python3.8/site-packages/sklearn/linear_model/_stochastic_gradient.py:713: ConvergenceWarning: Maximum number of iteration reached before convergence. Consider increasing max_iter to improve the fit.
  warnings.warn(


True